In [4]:
# Import necessary modules
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote
from sqlalchemy import text
import pandas as pd

In [7]:
# Define server name and database name
server_name   = "localhost"  # Change this to the actual server name
database_name = "BokhandelDB"   # Change this to the actual database name

# Construct connection string for SQL Server
connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
# Create URL string for SQLAlchemy engine
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

# Display the URL string after decoding
print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try: # Create engine and connect to the database  
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    # Handle connection error
    print('Error while connecting to database:\n')
    print(e)


Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=BokhandelDB;Trusted_Connection=yes 

Successfully connected to BokhandelDB!


In [9]:
def execute_stored_procedure(title, connection):
     # Define the query to execute the stored procedure
    query = text("""EXECUTE LetaBok :title""") 
      # Execute the stored procedure with the provided title parameter
    results = connection.execute(query, {'title': f'%{title}%'}).fetchall() 
    # Return the results
    return results

def search_book():  # Prompt user to enter a book title for search
    title = input("Search for a book: ").strip()
    
    # Connect to the database engine
    with engine.connect() as connection:
         # Execute the stored procedure to search for books
        results = execute_stored_procedure(title, connection) 
        
        # Convert the results into a DataFrame for easy manipulation
        df = pd.DataFrame(results, columns = ["Title", "Store", "Quantity"]) 
        
         # Display the number of results found
        print(f"Search for \"{title}\" gave {len(df)} results.")
        
          # If results are found, display them
        if len(df) > 0:
            display(df)            

In [13]:
# Function to search for a book title in the database and display results, handling potential misspellings
search_book()


Search for "Kafka" gave 1 results.


,Title,Store,Quantity
0,Kafka on the Shore,Boklunden,2
